In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Data Processing/Data for database/Maryland/maryland database drive mount/maryland_output'

/content/drive/.shortcut-targets-by-id/1pBbvU65l2gp7KNk7HfHrTBVuRp0GUffJ/Data Processing/Data for database/Maryland/maryland database drive mount/maryland_output


In [ ]:
!pip install turfpy
!pip install mysql.connector
!pip install geopandas
!pip install tqdm
!pip install gerrychain

In [4]:
from turfpy import measurement
from geojson import Point, Feature
from turfpy.transformation import intersect
from turfpy.measurement import length
import json
import mysql.connector
import geopandas as gpd
from tqdm import tqdm
from gerrychain import Graph
import shapely

In [5]:
"""
220_15 : 21
2_428 : 22
306_527 : 23
30_254 : 24
317_544 : 25
324_463 : 26
3_107 : 27
3_89 : 28
58_414: 29
8_74 : 30
"""

threads = [220,2,306,30,317,324,3,3,58,8]
rounds = [15,428,527,254,544,463,107,89,414,74]
dp_number = ['21','22','23','24','25','26','27','28','29','30']
state_number = '24'

In [6]:
for thread, round, districting_plan_number in zip(threads, rounds, dp_number):
  print(f'{thread}_{round} : {districting_plan_number}')

220_15 : 21
2_428 : 22
306_527 : 23
30_254 : 24
317_544 : 25
324_463 : 26
3_107 : 27
3_89 : 28
58_414 : 29
8_74 : 30


In [7]:
  censusblock_sql = "INSERT INTO census_block_kevin (\
  geoid20, \
  adjacent_congressional_district, \
  congressional_district, \
  congressional_districtid, \
  districting_plan, \
  precinctid, \
  state, \
  geometry_type, \
  geometry \
  ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"

In [8]:
for thread, round, districting_plan_number in zip(threads, rounds, dp_number):
  df = gpd.read_file(f'census block output/marylandRecomCB_{thread}_{round}.json')
  graph = Graph.from_geodataframe(df, ignore_errors=True)

  adjacency = list(graph.adjacency())
  for index in range(len(graph)):
    node_data = graph.nodes().data()[index]
    neighbors = adjacency[index][1]
    for neighbor, perim in neighbors.items():
      #print(neighbor, perim['shared_perim'])
      if node_data['CD116'] != graph.nodes.data()[neighbor]['CD116']:
        node_data['adjacent_district'] = graph.nodes.data()[neighbor]['CD116']
        break
    if 'adjacent_district' not in node_data:
      node_data['adjacent_district'] = None

  # SQL Code
  mydb = mysql.connector.connect(
    host='mysql3.cs.stonybrook.edu',
    user='MothBalls',
    password='changeit',
    database='MothBalls',
    connection_timeout=3600
  )
  mycursor = mydb.cursor()

  cb_vals = []
  for index in range(len(graph)):
    node_data = graph.nodes().data()[index]
    geometry = shapely.geometry.mapping(node_data['geometry'])
    cb_val = ((districting_plan_number + node_data['GEOID20']), \
        node_data['adjacent_district'], \
        node_data['CD116'], \
        (districting_plan_number + node_data['STATE'] + node_data['CD116']), \
        districting_plan_number, \
        node_data['PRECINCTID'],\
        node_data['STATE'], \
        geometry['type'], \
        json.dumps(geometry['coordinates']) \
    )
    cb_vals.append(cb_val)

  small_cb_vals = []
  i = 0
  for cb_val in tqdm(cb_vals):
    i += 1
    if i > 250:
      mycursor.executemany(censusblock_sql, small_cb_vals)
      small_cb_vals = []
      i = 0
    small_cb_vals.append(cb_val)
  mycursor.executemany(censusblock_sql, small_cb_vals)

  mydb.commit()
  mydb.close()

  print(f'{thread}_{round}: {districting_plan_number} complete')


/usr/local/lib/python3.7/dist-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/usr/local/lib/python3.7/dist-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
100%|██████████| 83827/83827 [02:18<00:00, 604.97it/s]


220_15: 21 complete


/usr/local/lib/python3.7/dist-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/usr/local/lib/python3.7/dist-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
100%|██████████| 83827/83827 [01:18<00:00, 1067.93it/s]


2_428: 22 complete


/usr/local/lib/python3.7/dist-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/usr/local/lib/python3.7/dist-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
100%|██████████| 83827/83827 [01:06<00:00, 1258.92it/s]


...

3_89: 28 complete


/usr/local/lib/python3.7/dist-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/usr/local/lib/python3.7/dist-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
100%|██████████| 83827/83827 [11:53<00:00, 117.53it/s]


58_414: 29 complete


/usr/local/lib/python3.7/dist-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/usr/local/lib/python3.7/dist-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
100%|██████████| 83827/83827 [01:33<00:00, 897.52it/s]


8_74: 30 complete
